In [57]:
import pandas as pd
import numpy as py
import torch
from torch import nn
from sklearn.metrics import accuracy_score
import csv

In [54]:
training_data = pd.read_csv('/home/kadenw/Workspace/kaggle/digit_recognizer/data/train.csv')
testing_data = pd.read_csv('/home/kadenw/Workspace/kaggle/digit_recognizer/data/test.csv')
submission = pd.read_csv('/home/kadenw/Workspace/kaggle/digit_recognizer/data/sample_submission.csv')
submission

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [44]:
training_data_tensor = torch.from_numpy(training_data.iloc[:, 1:].values)
training_data_tensor = training_data_tensor.to(torch.float32)
training_results_tensor = torch.from_numpy(training_data.iloc[:, 0].values)

In [45]:
testing_data_tensor = torch.from_numpy(testing_data.iloc[:, :].values)
testing_data_tensor = testing_data_tensor.to(torch.float32)

In [46]:
class digit_recognizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.double()
        self.fc1 = nn.Linear(in_features = 784,
                             out_features = 348)
        self.fc2 = nn.Linear(in_features = 348,
                             out_features = 68)
        self.fc3 = nn.Linear(in_features = 68,
                             out_features = 10)
        self.relu = nn.ReLU()
        self.soft = nn.Softmax(dim = 1)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [47]:
digit_recognizer = digit_recognizer()
loss_CEL = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(digit_recognizer.parameters(), lr=0.001)

In [49]:
epochs = 500
for epoch in range(epochs):
    digit_recognizer.train()
    pred = digit_recognizer(training_data_tensor)
    loss = loss_CEL(pred, training_results_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss}")

Epoch: 0 | Train loss: 9.043221473693848
Epoch: 100 | Train loss: 0.10991629213094711
Epoch: 200 | Train loss: 0.035489607602357864
Epoch: 300 | Train loss: 0.01272184494882822
Epoch: 400 | Train loss: 0.0054658567532896996


In [56]:
digit_recognizer.eval()
with torch.inference_mode():
    test_pred = digit_recognizer(testing_data_tensor)
    test_pred_np = test_pred.numpy()
    pred_num = test_pred_np.argmax(1)
pred_num

array([2, 0, 9, ..., 3, 9, 2])

In [60]:
submission['Label'] = pred_num
submission.to_csv('/home/kadenw/Workspace/kaggle/digit_recognizer/data/sample_submission.csv', index=False)